In [1]:
import dask
import dask.bag as db
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import pandas as pd
from pandas import DataFrame as df
import re
from datetime import datetime
import numpy as np
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

dask.config.set(scheduler='multiprocessing')
dask.config.set({"array.slicing.split_large_chunks": True})
    
%matplotlib inline

### Combine 3 Datasets

In [ ]:
'''
TODO:
    Bring datasets in with dask
    Convert to pandas
    Combine datasets
    Parse Code and Context
'''

# Regex to extract code blocks
* Find max length block and transfer to new column
* Take others and make context code column
* Check for "ERROR" and drop columns

## Parse main code block

In [37]:
def extract_code_block(body):
    pattern = r'<code>(.*?)</code>'
    match = re.findall(pattern, body)
    return "ERROR" if not match else str(max(match, key=len))

### Test

In [22]:
string1 = "Testing <code>long</code> something about this block <code>longer</code> xyz <code>longest</code>"
string2 = "Test without code"

In [38]:
df = pd.DataFrame()
df['body'] = [string1, string2]

In [39]:
df['block'] = df['body'].map(lambda x : extract_code_block(x))

In [40]:
df

,body,block
0,Testing <code>long</code> something about this...,longest
1,Test without code,ERROR


## Parse Context Code